In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

In [19]:
url = "https://www.99acres.com/"

def wait_time_loading(driver, wait):
    title = driver.title
    try:
        wait.until(
            lambda d: d.execute_script("return document.readyState") == "complete"  
        )
    except:
        print(f"the webpage {title} is not loaded")
    else:
        print(f"the webpage {title} did get loaded")





# options for making chrome look its legit and not a automation bot
chrome_options = Options()
chrome_options.add_argument("--disable-http2")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--enable-features=NetworkServiceInProcess")
chrome_options.add_argument("--disable-features=NetworkService")
#chrome_options.add_argument(
#    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36"
#)

driver = webdriver.Chrome(options=chrome_options)

driver.maximize_window()

wait = WebDriverWait(driver , 10)

driver.get(url)
wait_time_loading(driver, wait)






## for finding search bar and if fount thn clicking it

try:
    search_bar = wait.until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="keyword2"]'))
    )
except:
    print("Timeout error, the search bar element did not load")

else:
    search_bar.click()
    search_bar.send_keys("Bangalore")
    print("Element entered in search bar")
    time.sleep(1)


## clicking on the lsit after searching

try:
    list_element = wait.until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="0"]'))
    )
except:
    print("Timeout error, the element cannot be found")

else:
    list_element.click()
    print("list element is clicked")
    time.sleep(1)



## for searching the SEARCH button and clicking it

try:
    search_button = wait.until(
        EC.element_to_be_clickable((By.XPATH,'//*[@id="searchform_search_btn"]'))
    )
except:
    print("Timeout error, the search button is not clickable")

else:
    search_button.click()
    print("Successfully clicked on Search Button!!!")
    time.sleep(1)


## waiting for next page to load
wait_time_loading(driver, wait)


## finding the slicer button

try:
    slider = wait.until(
        EC.element_to_be_clickable((By.XPATH,'//*[@id="budgetLeftFilter_max_node"]'))
    )
except:
    print("Timeout error, the slicer is not clickable")
else:

    action= ActionChains(driver)
    (
        action.click_and_hold(slider).move_by_offset(-73,0).release().perform()
    )
    print("Slider moved")
time.sleep(2)


## 1. verified

verified = wait.until(
	EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[4]/div[3]/div[1]/div[3]/section/div/div/div/div/div[1]/div/div[3]/span[2]'))
)

verified.click()
time.sleep(2)




## 2.Ready to mover
ready = wait.until(
	EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/div[4]/div[3]/div[1]/div[3]/section/div/div/div/div/div[1]/div/div[5]/span[2]'))
)
ready.click()

## load filters
while True:
    try:
        filter_button = wait.until(
            EC.element_to_be_clickable((By.XPATH,"//i[@class='iconS_Common_24 icon_upArrow cc__rightArrow']"))
        )
    except:
        print("timeout, all the filtered are uncoverd")
        break
    else:
        filter_button.click()



## 3. photos and videos

photo = wait.until(
    EC.element_to_be_clickable((By.XPATH,'/html/body/div[1]/div/div/div[4]/div[3]/div[1]/div[3]/section/div/div/div/div/div[2]/div/div[6]/span[2]'))
)
photo.click()
time.sleep(1)


videos = wait.until(
    EC.element_to_be_clickable((By.XPATH,'/html/body/div[1]/div/div/div[4]/div[3]/div[1]/div[3]/section/div/div/div/div/div[2]/div/div[7]/span[2]'))
)
videos.click()

# Navigate pages and extract data
data = []
page_count = 0

## Scroll and collect data until "Next Page >" button appears
pagination_ended = False

while not pagination_ended:
    page_count += 1
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

    
    # Extract new listings
    rows = driver.find_elements(By.CLASS_NAME, "tupleNew__TupleContent")
    for row in rows:
        # property name
        try:
            name = row.find_element(By.CLASS_NAME, "tupleNew__headingNrera").text
        except:
            name = np.nan

        # property location
        try:
            location = row.find_element(By.CLASS_NAME, "tupleNew__propType").text
        except:
            location = np.nan

        # property price
        try:
            price = row.find_element(By.CLASS_NAME, "tupleNew__priceValWrap").text
        except:
            price = np.nan

        # property area and bhk
        try:
            elements = row.find_elements(By.CLASS_NAME, "tupleNew__area1Type")
        except:
            area, bhk = [np.nan, np.nan]
        else:
            area, bhk = [ele.text for ele in elements]

        property = {
            "name": name,
            "location": location,
            "price": price,
            "area": area,
            "bhk": bhk
        }
        data.append(property)
    print(f"Page {page_count} scraped")
    time.sleep(3)

    # Check if "Next Page >" button exists
    next_buttons = driver.find_elements(By.XPATH, "/html/body/div[1]/div/div/div[4]/div[3]/div[3]/div[4]/a")
    
    if next_buttons:
        print("Reached pagination button. Switching to button-click mode.")
        print("-----------------------")

        try:
            next_btn = wait.until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div/div/div[4]/div[3]/div[3]/div[4]/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({behavior:'smooth', block:'center'});", next_btn)
            time.sleep(2)
            next_btn.click()
            time.sleep(3)
        except:
            print(" Pagination click failed. Exiting loop.")
            pagination_ended = True
        else:
            print(f"Clicked {page_count +1} next page.")

    
            
    else:
        print("No more 'Next Page' button. Scraping done.")
        pagination_ended = True

        
time.sleep(2)
#driver.quit()




the webpage India Real Estate Property Site - Buy Sell Rent Properties Portal - 99acres.com did get loaded
Element entered in search bar
list element is clicked
Successfully clicked on Search Button!!!
the webpage Property in Bangalore - Real Estate in Bangalore did get loaded
Slider moved
timeout, all the filtered are uncoverd
Page 1 scraped
Reached pagination button. Switching to button-click mode.
-----------------------
Clicked 2 next page.
Page 2 scraped
Reached pagination button. Switching to button-click mode.
-----------------------
Clicked 3 next page.
Page 3 scraped
Reached pagination button. Switching to button-click mode.
-----------------------
Clicked 4 next page.
Page 4 scraped
Reached pagination button. Switching to button-click mode.
-----------------------
Clicked 5 next page.
Page 5 scraped
Reached pagination button. Switching to button-click mode.
-----------------------
Clicked 6 next page.
Page 6 scraped
Reached pagination button. Switching to button-click mode.
-

In [26]:

data=pd.DataFrame(data)
data

,name,location,price,area,bhk
0,Sobha Royal Pavilion\n4.2,"3 BHK Flat in Sarjapur Road, Bangalore",₹2.4 Cr,"1,507 sqft",3 BHK
1,Sobha Royal Pavilion\n4.2,"3 BHK Flat in Carmelaram, Sarjapur Road",₹2.88 Cr,"1,800 sqft",3 BHK
2,Sobha Windsor\n4.3,"3 BHK Flat in Whitefield, Bangalore",₹2.1 Cr,"1,550 sqft",3 BHK
3,"Vajarahalli, Bangalore, Bangalore South","4 Bedroom House in Vajarahalli, Bangalore",₹4.45 Cr,"1,500 sqft",4 BHK
4,Sobha Royal Pavilion\n4.2,"4 BHK Flat in Sarjapur Road, Bangalore",₹3.8 Cr,"2,232 sqft",4 BHK
...,...,...,...,...,...
407,Prestige Jindal City\n4.1,"2 BHK Flat in Tumkur Road, Bangalore",₹1.25 Cr,962 sqft,2 BHK
408,Prestige Primrose Hills,"2 BHK Flat in Uttarahalli Manavarthekaval, Kan...",₹1 Cr,988 sqft,2 BHK
409,Salarpuria Silver Woods,"3 BHK Flat in Nagavarapalya, Old Madras Road",₹2.55 Cr,"1,813 sqft",3 BHK
410,Samruddhi,"3 BHK Flat in Uttarahalli, Bangalore",₹95 Lac,"1,320 sqft",3 BHK


In [30]:
data.to_excel("bengaluru-properties-99acres(Uncleaned_data).xlsx", index=False)


In [31]:

df_properties = (
	pd
	.DataFrame(data)
	.drop_duplicates()
	.apply(lambda col: col.str.strip().str.lower() if col.dtype == "object" else col)
	.assign(
		is_starred=lambda df_: df_.name.str.contains("\n").astype(int),
		name=lambda df_: (
			df_
			.name
			.str.replace("\n[0-9.]+", "", regex=True)
			.str.strip()
			.replace("adroit district s", "adroit district's")
		),
		location=lambda df_: (
			df_
			.location
			.str.replace("bangalore", "")
			.str.strip()
			.str.replace(",$", "", regex=True)
			.str.split("in")
			.str[-1]
			.str.strip()
		),
		price=lambda df_: (
			df_
			.price
			.str.replace("₹", "")
			.apply(lambda val: float(val.replace("lac", "").strip()) if "lac" in val else float(val.replace("cr", "").strip()) * 100)
		),
		area=lambda df_: (
			df_
			.area
			.str.replace("sqft", "")
			.str.strip()
			.str.replace(",", "")
			.pipe(lambda ser: pd.to_numeric(ser))
		),
		bhk=lambda df_: (
			df_
			.bhk
			.str.replace("bhk", "")
			.str.strip()
			.pipe(lambda ser: pd.to_numeric(ser))
		)
	)
	.rename(columns={
		"price": "price_lakhs",
		"area": "area_sqft"
	})
	.reset_index(drop=True)
)



In [32]:
df_properties.to_excel("bengaluru-properties-99acres.xlsx", index=False)


In [35]:
df_properties

,name,location,price_lakhs,area_sqft,bhk,is_starred
0,sobha royal pavilion,sarjapur road,240.0,1507,3,1
1,sobha royal pavilion,"carmelaram, sarjapur road",288.0,1800,3,1
2,sobha windsor,whitefield,210.0,1550,3,1
3,"vajarahalli, bangalore, bangalore south",vajarahalli,445.0,1500,4,0
4,sobha royal pavilion,sarjapur road,380.0,2232,4,1
...,...,...,...,...,...,...
393,prestige jindal city,tumkur road,125.0,962,2,1
394,prestige primrose hills,"uttarahalli manavarthekaval, kanakapura road",100.0,988,2,0
395,salarpuria silver woods,"nagavarapalya, old madras road",255.0,1813,3,0
396,samruddhi,uttarahalli,95.0,1320,3,0
